Task 2: Implement a program which
1.   for each **unique labe**l l, computes the correspending **c most significant clusters** associated with the even numbered Caltec101 images (using DBScan algorithm); the resulting clusters should be visualized both

*   as differently colored point clouds in a 2-dimensional **MDS** space, and
*   as **groups of image thumbnails**. and

2.   for the odd numbered images, predicts the most likely labels using the c label-specific clusters.

The system should also output per-label precision, recall, and F1-score values as well as output an overall accuracy
value.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports
import numpy as np
import pandas as pd
import pickle

import json
import torch
from torchvision.datasets import Caltech101
from torchvision.models  import resnet50, ResNet50_Weights
from torchvision import transforms

import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean, cosine, minkowski, correlation
from PIL import Image
from sklearn.preprocessing import  minmax_scale
from sklearn.cluster import DBSCAN

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import collections
from collections import defaultdict


In [ ]:
# path to drive folder
path = '/content/drive/MyDrive/CSE515_Phase3'

In [ ]:
# Load the caltech101 dataset
data = Caltech101(root = f'{path}/data', download = True)

Files already downloaded and verified


In [ ]:
# Label image mapping
with open(f'{path}/label_image_map.json','r') as fp:
    label_image_map = json.load(fp)

In [ ]:
# Latent space SVD layer 3
with open(f'{path}/latent_spaces/SVD_66_layer_3_latent_weights.json','r') as fp:
    latent_features = json.load(fp)

In [ ]:
# Methods to calculate per label metrics of Precision, Recall, F1 score and Accuracy
def calculate_per_label_metrics(actual_labels, predicted_labels):
    unique_labels = set(actual_labels + predicted_labels)
    label_to_index = {label: i for i, label in enumerate(unique_labels)}
    label_count = len(unique_labels)

    true_positives = [0] * label_count
    false_positives = [0] * label_count
    false_negatives = [0] * label_count

    for actual, predicted in zip(actual_labels, predicted_labels):
        actual_index = label_to_index[actual]
        predicted_index = label_to_index[predicted]

        if actual == predicted:
            true_positives[actual_index] += 1
        else:
            false_positives[predicted_index] += 1
            false_negatives[actual_index] += 1

    precision = [true_positives[i] / (true_positives[i] + false_positives[i] + 1e-10) for i in range(label_count)]
    recall = [true_positives[i] / (true_positives[i] + false_negatives[i] + 1e-10) for i in range(label_count)]
    f1_score = [2 * (p * r) / (p + r + 1e-10) for p, r in zip(precision, recall)]

    return precision, recall, f1_score

def calculate_accuracy(actual_labels, predicted_labels):
    correct_predictions = sum(1 for a, p in zip(actual_labels, predicted_labels) if a == p)
    accuracy = correct_predictions / len(actual_labels)
    return accuracy

In [ ]:
# Multi Dimensional Scaling and stress calculation
def stress_function(original_distances, reduced_distances):
    stress = np.sqrt(((reduced_distances.ravel() - original_distances.ravel()) ** 2).sum() / ((original_distances.ravel() ** 2).sum()))
    return stress

def MDS(data, num_dimensions):
    num_images, num_features = len(data), len(data[0])
    # Random Positions
    X = np.random.rand(num_images, num_dimensions)
    # Set optimization parameters
    max_iterations = 300

    original_distances = euclidean_distances(data)
    original_distances[original_distances == 0] = 1e-8

    # Optimization loop
    for iteration in range(max_iterations):
        # Compute the current stress value
        reduced_distances = euclidean_distances(X)
        current_stress = stress_function(original_distances, reduced_distances)
        if current_stress < 0.05:
            break
        # Guttman Transformation
        reduced_distances[reduced_distances == 0] = 1e-8
        ratio = original_distances / reduced_distances
        B = -ratio
        B[np.arange(len(B)), np.arange(len(B))] += ratio.sum(axis=1)
        X = 1.0 / original_distances.shape[0] * np.dot(B, X)

        reduced_distances = np.sqrt((X**2).sum(axis=1)).sum()

    return X, original_distances


In [ ]:
# DBSCAN Methods

def get_core_point(distance, epsilon, M):
    density_dict = {}
    core_point = []
    for index, node in enumerate(distance):
        epsilon_neib = np.squeeze(np.argwhere(node < epsilon))
        #  epsilon
        if epsilon_neib.size == 0:
            node_density_set = []
        #  epsilon
        elif epsilon_neib.size == 1:
            node_density_set = [int(epsilon_neib)]
        #  epsilon
        else:
            node_density_set = list(epsilon_neib)
            if epsilon_neib.size >= M:
                core_point.append(index)
        density_dict[index] = node_density_set
    return density_dict, core_point

def get_noise_point(density_dict, core_point, data_length):
    noncore_point = set(range(data_length)) - set(core_point)
    noise_point = []
    for point in noncore_point:
        if len(set(density_dict[point]) & set(core_point)) == 0:
            noise_point.append(point)
    return noise_point

def assign_class(density_dict, core_point, data_length, class_list, noise_point):
    # core_point.reverse()
    cluster_quant = {}
    for core in core_point:
        if class_list[core] == -1:
            class_list[core] = core
            cluster_quant[class_list[core]]=1
            density_propagation(density_dict, core, class_list, core_point, noise_point, cluster_quant)
    sorted_dict = dict(sorted(cluster_quant.items(), key=lambda item: item[1]))
    for k in sorted_dict.keys():
        visited = []
        # print(k)
        for epsilon_neib in density_dict[k]:
            density_propagation_non_core(density_dict, epsilon_neib, class_list, core_point, visited)
    return class_list
# Recurrsion method to identify classes and non core points
def density_propagation_non_core(density_dict, core, class_list, core_point, visited):
    visited.append(core)
    for epsilon_neib in density_dict[core]:
      if epsilon_neib in core_point and epsilon_neib not in visited:
        density_propagation_non_core(density_dict, epsilon_neib, class_list, core_point, visited)
      elif class_list[epsilon_neib] == -1:
        class_list[epsilon_neib] = class_list[core]

# Recurrsion method to identify classes
def density_propagation(density_dict, core, class_list, core_point, noise_point, cluster_quant):
    # core_point.reverse()
    for epsilon_neib in density_dict[core]:
        if epsilon_neib and class_list[epsilon_neib] == -1:
            if epsilon_neib in core_point:
              class_list[epsilon_neib] = class_list[core]
              cluster_quant[class_list[core]]+=1
              density_propagation(density_dict, epsilon_neib, class_list, core_point, noise_point, cluster_quant)


def show_result(class_list, raw_data):
    colors = [
              '#FF0000', '#FFA500', '#FFFF00', '#00FF00', '#228B22',
              '#0000FF', '#FF1493', '#EE82EE', '#000000', '#FFA500',
              '#00FF00', '#006400', '#00FFFF', '#0000FF', '#FFFACD',
              ]

    use_color = {}
    total_color = list(dict(collections.Counter(class_list)).keys())
    if -1 in total_color:
        total_color.remove(-1)
    for index, i in enumerate(total_color):
        use_color[i] = index
    plt.figure(num=1, figsize=(15, 10))
    for node, class_ in enumerate(class_list):
        if class_ != -1:
            plt.scatter(x=raw_data[node,0], y=raw_data[node,1], s=5, marker='o', alpha=0.73, c = colors[use_color[class_]])
        else:
            plt.scatter(x=raw_data[node,0], y=raw_data[node,1], c='b', s=20, marker='+',
                        alpha=0.8)
    plt.title('The Result Of Cluster')
    plt.show()

# Main dbscan method
def DBSCAN_main(distances, mds_data, epsilon, min_count):
    data_length = distances.shape[0]
    class_list = [-1 for _ in range(data_length)]

    density_dict, core_point = get_core_point(distances, epsilon, min_count)

    #print('Core points: ',len(core_point))
    noise_point = get_noise_point(density_dict, core_point, data_length)
    #print('Noise points: ',len(noise_point))
    class_list = assign_class(density_dict, core_point, data_length, class_list, noise_point)
    #print('Classes: ',len(class_list),set(class_list))

    return class_list, core_point, len(noise_point)

In [31]:
# Display images thumbnails of the core points
def display_images_in_grid(label, core_points, columns=5, image_size=(2, 2), spacing=(0.2, 0.2)):
    rows = (len(core_points) + columns - 1) // columns
    if rows==1:
        columns = len(core_points)
    fig, axes = plt.subplots(rows, columns, figsize=(columns * image_size[0], rows * image_size[1]))
    plt.subplots_adjust(wspace=spacing[0], hspace=spacing[1])

    for i, idx in enumerate(core_points):
      img = data[label_image_map[label][idx]][0]

      if rows == 1:
        axes[i].imshow(img)
        axes[i].set_title(class_list[idx], fontsize=8)
        axes[i].axis('off')
      else:
        row, col = divmod(i, columns)
        axes[row, col].imshow(img)
        axes[row, col].set_title(class_list[idx], fontsize=8)
        axes[row, col].axis('off')

    # Hide any remaining empty subplots
    for i in range(len(core_points), rows * columns):
      if rows == 1:
            axes[i].axis('off')
      else:
        row, col = divmod(i, columns)
        axes[row, col].axis('off')

    plt.show()


In [ ]:
# Load the precalculated clusters and label DBSCAN parameters
def load_clusters(target_clusters):
    with open(f'{path}/clusters{target_clusters}.json','r') as fp:
        clusters = json.load(fp)

    with open(f'{path}/labels_dbscan_data{target_clusters}.json','r') as fp:
        labels_dbscan_param = json.load(fp)

    return clusters, labels_dbscan_param

In [ ]:
# for _ in core_point:
#     if _ ==-1:
#         continue
#     fig= plt.figure(figsize=(15, 5))
#     for ix,i in enumerate(class_list[_]):
#         fig.add_subplot(2, 6,ix+1)
#         plt.imshow(data[names[i]][0])
#         plt.title(f'cluster core:{_}')
#         plt.axis('off')

In [ ]:
# New Image Label Prediction

In [ ]:
# Get the Resnet50 output for an image by hooks in intermediate layers: layer3, avgpool, fc
def get_resnet(image):

    # Load the model resnet50
    with torch.no_grad():
        m = resnet50(pretrained=True)

        # Preprocess the image into 224x224 and to Tensor
        preprocess = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
        tensor_image = preprocess(image)

        if tensor_image.shape[0]==1:
            tensor_image = torch.cat([tensor_image,tensor_image,tensor_image], dim=0)
        outputs = {}

        # Create hook function to get output of particular layers in resnet50
        def wrap_hook(name): # wrap the hook function to get layer name as key in the output dictionary
            def hook(module, input, output):
                outputs[name] = output
            return hook

        # Layers to be hooked
        layers = {'layer3': m.layer3,'avgpool': m.avgpool,'fc': m.fc}

        for name,layer in layers.items():
            layer.register_forward_hook(wrap_hook(name)) # Register the hook

        # Pass the image in the pretrained model
        model = m(tensor_image.unsqueeze(0))

        # Get the features and reshape to required dimensions
        features_avgpool = outputs['avgpool'].reshape((1024,2)).mean(dim=[1])
        features_layer3 = outputs['layer3'].reshape((1024,14,14)).mean(dim=[1,2])
        features_fc = outputs['fc'].reshape(-1)

    return [features_avgpool,features_layer3,features_fc]


In [ ]:
# Transform the odd image to SVD latent space
def transform_odd_images_to_latent_space(latent_space, image_feature_vector):
    transformed_latent_space_vector = np.dot(np.array(image_feature_vector).reshape(1,-1), np.array(latent_space["V"]))
    for i in range(transformed_latent_space_vector.shape[1]):
        transformed_latent_space_vector[:, i] /= latent_space["sigma"][i]
    return transformed_latent_space_vector.squeeze()

In [ ]:
# SVD layer3 matrices: V, Sigma
with open(f"{path}/latent_spaces/SVD_66_layer_3_latent_space.json", "r") as fp:
    latent_space = json.load(fp)

with open(f'{path}/classification_list.json','r') as fp:
    classification_list = json.load(fp)

# Show the calculated classification metrics
def calculate_classification_metrics(actual_labels, predicted_labels):
    unique_labels = list(range(101))
    per_label_precision, per_label_recall, per_label_f1_score = calculate_per_label_metrics(actual_labels, predicted_labels)
    for label, precision, recall, f1 in zip(unique_labels, per_label_precision, per_label_recall, per_label_f1_score):
        print(f"Label {label}: Precision={precision:.2f}, Recall={recall:.2f}, F1 Score={f1:.2f}")

    overall_accuracy = calculate_accuracy(actual_labels, predicted_labels)
    print(f"Over all accuracy for the model is = {round(overall_accuracy*100,2)} %")

In [32]:
# Main code to get clusters for labels and image thumbnails
target_clusters = int(input('Give target c clusters, 5 or 10: '))
clusters, labels_dbscan_param = load_clusters(target_clusters)
# For all Labels
for label in label_image_map:
    l = []
    for i in label_image_map[label]:
        l.append(latent_features[str(i)])
    l = np.array(l)
    print(f'For label {label}, feature matrix shape {l.shape}')
    min_count, epsilon = labels_dbscan_param[label]

    mds_data, distances = MDS(l, 2)

    # Main DBSCAN from scratch function distances, mincount_low, mincount_high, epsilon_low, epsilon_high, target_clusters, max_itr=100
    class_list, core_point, noise_cnt = DBSCAN_main(distances, mds_data, epsilon, min_count)

    num_clusters = len(set(class_list)) - (1 if -1 in class_list else 0)
    print(f"Saved values for label:{label}, epsilon:{epsilon}, mincount:{min_count}, Clusters:{num_clusters}, core points:{len(core_point)}")

    #use mds projected data in 2d to display the current clusters obtained via dbscan
    show_result(class_list, mds_data)
    display_images_in_grid(label, core_point)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
# Main code to get label predictions for Odd Image
# This code will calculate the resnet features for odd images and find distance of the image to core points of the labels, and pick one most likely label
if input('Do you want 1. Saved file, 2. Calculate file again (Takes Time): ') == '2':
    classification_list = {}
    for image_id in range(1,8677,2):
        input_image = image_id
        image = data[input_image][0]

        image_avgpool,image_layer3,image_fc = get_resnet(image)
        image_layer3 = np.array(image_layer3)

        image_latent_features = transform_odd_images_to_latent_space(latent_space, image_layer3)
        labels = []
        for label in label_image_map:
            epsilon = labels_dbscan_param[label][1]
            core_points = clusters[label]['core']
            connected = 0
            min_dist = 100
            for core in core_points:
                a = image_latent_features
                b = latent_features[str(core)]
                dist = euclidean(a,b)
                if dist<=epsilon:
                    connected+=1
                min_dist = min(dist,min_dist)
            labels.append((label,min_dist,connected))
        actual_label = data[image_id][1]
        labels = sorted(labels, key = lambda x: x[1])
        classification_list[image_id] = [int(labels[0][0]), actual_label]
    with open(f'{path}/classification_list.json','w') as fp:
        json.dump(classification_list,fp)

actual_labels, predicted_labels = [], []
for i in classification_list:
    lab,alab = classification_list[i]
    predicted_labels.append(lab)
    actual_labels.append(alab)
# Calculate and show the classification metrics
calculate_classification_metrics(actual_labels, predicted_labels)


Do you want 1. Saved file, 2. Calculate file again (Takes Time): 1
Label 0: Precision=0.93, Recall=0.99, F1 Score=0.96
Label 1: Precision=0.98, Recall=0.94, F1 Score=0.96
Label 2: Precision=0.88, Recall=1.00, F1 Score=0.94
Label 3: Precision=0.87, Recall=0.97, F1 Score=0.92
Label 4: Precision=0.68, Recall=0.85, F1 Score=0.75
Label 5: Precision=0.99, Recall=0.42, F1 Score=0.59
Label 6: Precision=0.08, Recall=0.10, F1 Score=0.09
Label 7: Precision=0.10, Recall=0.14, F1 Score=0.12
Label 8: Precision=1.00, Recall=0.50, F1 Score=0.67
Label 9: Precision=0.15, Recall=0.15, F1 Score=0.15
Label 10: Precision=0.20, Recall=0.22, F1 Score=0.21
Label 11: Precision=0.43, Recall=0.62, F1 Score=0.51
Label 12: Precision=0.83, Recall=0.69, F1 Score=0.75
Label 13: Precision=0.88, Recall=0.59, F1 Score=0.71
Label 14: Precision=0.29, Recall=0.23, F1 Score=0.26
Label 15: Precision=0.62, Recall=0.38, F1 Score=0.47
Label 16: Precision=0.56, Recall=0.30, F1 Score=0.39
Label 17: Precision=0.77, Recall=0.68, F1 

In [ ]:
'''
def save_dbscan_results(clusters, dbscan_data,target_clusters):
  # Save Clusters into json
  with open(f'{path}/clusters{target_clusters}.json','w') as fp:
      json.dump(clusters,fp)
  # Save MinCount Epsilon into json
  with open(f'{path}/labels_dbscan_data{target_clusters}.json','w') as fp:
      json.dump(dbscan_data,fp)

def binary_search_clusters(distances, mincount_low, mincount_high, epsilon_low, epsilon_high, target_clusters, max_itr=500):
  best_noise = distances.shape[0]
  best_clusters = 0
  while mincount_low <= mincount_high:
    itr = 0
    ep_l = epsilon_low
    ep_h = epsilon_high
    while itr<=max_itr:
        mid_epsilon = (epsilon_low + epsilon_high) / 2
        #print(f"Epsilon:{mid_epsilon}, mincount:{mincount_high}")
        class_list, core_point, noise_cnt = DBSCAN_main(distances, mds_data, epsilon = mid_epsilon, min_count = mincount_high)
        num_clusters = len(set(class_list)) - (1 if -1 in class_list else 0)
        itr+=1
        if num_clusters==target_clusters:
            print('best found at :',mincount_high, mid_epsilon)
        if num_clusters > best_clusters:
            best_clusters = num_clusters
            best_noise = noise_cnt
            best_eps = mid_epsilon
            best_mincount = mincount_high
        if num_clusters==0 or num_clusters > target_clusters or len(core_point) < target_clusters:
            epsilon_low = mid_epsilon
        else:
            epsilon_high = mid_epsilon - 1e-6

    # Reset epsilon range for the next iteration
    epsilon_low, epsilon_high = ep_l, ep_h
    mincount_high -= 1

  # # If no exact match is found, return the closest values
  # show_result(class_list, mds_data)
  print('noise: ',best_noise)
  return best_mincount, best_eps, class_list, core_point
'''

In [ ]:
'''target_clusters = int(input('Give target c clusters: '))
# For all Labels
for label in label_image_map:
    l = []
    for i in label_image_map[label]:
        l.append(latent_features[str(i)])
    l = np.array(l)
    print(f'For label {label}, feature matrix shape {l.shape}')

    mds_data, distances = MDS(l, 2)

    #use mds projected data in 2d to display the current clusters obtained via dbscan
    show_result(class_list, mds_data)

    # Store MinPoint, Epsilon values and scale
    labels_dbscan_param[label] = [min_count, epsilon]

    # Get Clusters from class list and store
    clusters[label] = {'core':[], 'border':[]}

    for point, class_ in enumerate(class_list):
      if point in core_point:
        clusters[label]['core'].append(label_image_map[str(label)][point])
      elif class_list != -1:
        clusters[label]['border'].append(label_image_map[str(label)][point])

save_dbscan_results(clusters, labels_dbscan_param,target_clusters)'''